<a href="https://colab.research.google.com/github/pranao0609/DL_Projects/blob/main/24_Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
df = pd.read_csv(url)[['label', 'tweet']]
df.columns = ['label', 'text']

In [23]:
# ✅ Binary classification (hate = 1, non-hate = 0)
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)
vectorizer = CountVectorizer(max_features=3000)
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# ✅ Convert to torch tensors
X_train_tensor = torch.tensor(X_train_vec, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_vec, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)


In [24]:
class HateSpeechModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, x):
        return self.fc(x)

model = HateSpeechModel(X_train_tensor.shape[1])
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
for epoch in range(4):
    outputs = model(X_train_tensor)
    loss = loss_fn(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 1.1246
Epoch 2, Loss: 1.1077
Epoch 3, Loss: 1.0910
Epoch 4, Loss: 1.0740


In [26]:
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)
    predicted = torch.argmax(preds, 1)
    acc = (predicted == y_test_tensor).float().mean()
    print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 0.4098
